In [1]:
# Step 0: Install Required Tools and Libraries
# Update package list
!apt-get update

# Install build tools and SRA Toolkit
!apt-get install -y build-essential sra-toolkit unzip

# Download and compile STAR
!wget https://github.com/alexdobin/STAR/archive/2.7.10a.zip
!unzip 2.7.10a.zip
!cd STAR-2.7.10a/source && make STAR


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,437 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,526 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,826 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [38.5 kB]
Get:14 h

In [6]:
# Step 1: Download FASTQ Files from SRA
# Download and convert SRR11548957 to paired-end FASTQ files
!prefetch SRR11548957
!fasterq-dump SRR11548957 --split-files


2025-11-11T05:14:13 prefetch.2.11.3: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2025-11-11T05:14:14 prefetch.2.11.3: 1) 'SRR11548957' is found locally
2025-11-11T05:14:14 prefetch.2.11.3: 'SRR11548957' has 0 unresolved dependencies
spots read      : 1,792,464
reads read      : 3,584,928
reads written   : 3,584,928


In [11]:
# Step 2: Prepare Reference Genome (chr22)
# Create reference directory and download chr22 from Ensembl
!mkdir -p reference
!wget -P reference ftp://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz
!wget -P reference ftp://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz

# Unzip the FASTA and GTF files
!gunzip reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz
!gunzip reference/Homo_sapiens.GRCh38.109.gtf.gz

--2025-11-11 05:16:00--  ftp://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz
           => ‘reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-109/fasta/homo_sapiens/dna ... done.
==> SIZE Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz ... 11389810
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz ... done.
Length: 11389810 (11M) (unauthoritative)

Homo_sapiens.GRCh38 100%[===================>]  10.86M  9.30MB/s    in 1.2s    

2025-11-11 05:16:03 (9.30 MB/s) - ‘reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz’ saved [11389810]

--2025-11-11 05:16:03--  ftp://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz
 

In [12]:
#  Step 3: Generate STAR Genome Index
# Create index directory and build genome index
!mkdir -p star_index
!/content/STAR-2.7.10a/bin/Linux_x86_64_static/STAR \
    --runThreadN 4 \
    --runMode genomeGenerate \
    --genomeDir star_index \
    --genomeFastaFiles reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa \
    --sjdbGTFfile reference/Homo_sapiens.GRCh38.109.gtf \
    --genomeSAindexNbases 9

	/content/STAR-2.7.10a/bin/Linux_x86_64_static/STAR --runThreadN 4 --runMode genomeGenerate --genomeDir star_index --genomeFastaFiles reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa --sjdbGTFfile reference/Homo_sapiens.GRCh38.109.gtf --genomeSAindexNbases 9
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 11 05:16:36 ..... started STAR run
Nov 11 05:16:36 ... starting to generate Genome files
Nov 11 05:16:37 ..... processing annotations GTF
Nov 11 05:16:44 ... starting to sort Suffix Array. This may take a long time...
Nov 11 05:16:44 ... sorting Suffix Array chunks and saving them to disk...
Nov 11 05:16:53 ... loading chunks from disk, packing SA...
Nov 11 05:16:54 ... finished generating suffix array
Nov 11 05:16:54 ... generating Suffix Array index
Nov 11 05:16:54 ... completed Suffix Array index
Nov 11 05:16:54 ..... inserting junctions into the genome indices
Nov 11 05:16:58 ... writing Genome to disk ...
Nov 1

In [13]:
#  Step 4: Run STAR Alignment
# Create output directory and align reads to reference genome
!mkdir -p star_output
!/content/STAR-2.7.10a/bin/Linux_x86_64_static/STAR \
    --runThreadN 4 \
    --genomeDir star_index \
    --readFilesIn SRR11548957_1.fastq SRR11548957_2.fastq \
    --outFileNamePrefix star_output/SRR11548957_ \
    --outSAMtype BAM SortedByCoordinate \
    --quantMode GeneCounts

	/content/STAR-2.7.10a/bin/Linux_x86_64_static/STAR --runThreadN 4 --genomeDir star_index --readFilesIn SRR11548957_1.fastq SRR11548957_2.fastq --outFileNamePrefix star_output/SRR11548957_ --outSAMtype BAM SortedByCoordinate --quantMode GeneCounts
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 11 05:17:18 ..... started STAR run
Nov 11 05:17:18 ..... loading genome
Nov 11 05:17:18 ..... started mapping
Nov 11 05:39:57 ..... finished mapping
Nov 11 05:39:58 ..... started sorting BAM
Nov 11 05:39:58 ..... finished successfully


In [14]:
#  Step 5: Inspect Output Files
# List key output files
!ls -lh star_output/SRR11548957_Aligned.sortedByCoord.out.bam
!ls -lh star_output/SRR11548957_ReadsPerGene.out.tab
!cat star_output/SRR11548957_Log.final.out


-rw-r--r-- 1 root root 19M Nov 11 05:39 star_output/SRR11548957_Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 root root 32K Nov 11 05:39 star_output/SRR11548957_ReadsPerGene.out.tab
                                 Started job on |	Nov 11 05:17:18
                             Started mapping on |	Nov 11 05:17:18
                                    Finished on |	Nov 11 05:39:58
       Mapping speed, Million of reads per hour |	4.74

                          Number of input reads |	1792464
                      Average input read length |	150
                                    UNIQUE READS:
                   Uniquely mapped reads number |	156947
                        Uniquely mapped reads % |	8.76%
                          Average mapped length |	141.22
                       Number of splices: Total |	12073
            Number of splices: Annotated (sjdb) |	6261
                       Number of splices: GT/AG |	10520
                       Number of splices: GC/AG |	209
              

In [15]:
# Step 6: Download Output Files (Colab)
from google.colab import files
files.download('star_output/SRR11548957_Aligned.sortedByCoord.out.bam')
files.download('star_output/SRR11548957_ReadsPerGene.out.tab')
files.download('star_output/SRR11548957_Log.final.out')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task
Generate a Plotly plot of the STAR alignment results from the log file "star_output/SRR11548957_Log.final.out".

## Parse log file

### Subtask:
Read the `star_output/SRR11548957_Log.final.out` file and extract the relevant mapping statistics (e.g., unique mapping percentage, multi-mapping percentage, unmapped percentages).


**Reasoning**:
I need to read the log file and extract the relevant mapping statistics to prepare for plotting. I will use Python to open and parse the file, extracting the required percentages.



In [16]:
import pandas as pd

log_file_path = 'star_output/SRR11548957_Log.final.out'
mapping_stats = {}

with open(log_file_path, 'r') as f:
    for line in f:
        if 'Uniquely mapped reads %' in line:
            mapping_stats['Uniquely mapped reads %'] = float(line.split('|')[-1].strip().replace('%', ''))
        elif '% of reads mapped to multiple loci' in line:
            mapping_stats['% of reads mapped to multiple loci'] = float(line.split('|')[-1].strip().replace('%', ''))
        elif '% of reads mapped to too many loci' in line:
            mapping_stats['% of reads mapped to too many loci'] = float(line.split('|')[-1].strip().replace('%', ''))
        elif '% of reads unmapped: too many mismatches' in line:
            mapping_stats['% of reads unmapped: too many mismatches'] = float(line.split('|')[-1].strip().replace('%', ''))
        elif '% of reads unmapped: too short' in line:
            mapping_stats['% of reads unmapped: too short'] = float(line.split('|')[-1].strip().replace('%', ''))
        elif '% of reads unmapped: other' in line:
            mapping_stats['% of reads unmapped: other'] = float(line.split('|')[-1].strip().replace('%', ''))

print(mapping_stats)

{'Uniquely mapped reads %': 8.76, '% of reads mapped to multiple loci': 1.66, '% of reads mapped to too many loci': 0.08, '% of reads unmapped: too many mismatches': 0.0, '% of reads unmapped: too short': 89.38, '% of reads unmapped: other': 0.12}


## Prepare data for plotting

### Subtask:
Organize the extracted statistics into a format suitable for Plotly. A pandas DataFrame would be ideal.


**Reasoning**:
Convert the mapping statistics dictionary into a pandas DataFrame with 'Category' and 'Percentage' columns as required.



In [17]:
mapping_df = pd.DataFrame(list(mapping_stats.items()), columns=['Category', 'Percentage'])
display(mapping_df)

,Category,Percentage
0,Uniquely mapped reads %,8.76
1,% of reads mapped to multiple loci,1.66
2,% of reads mapped to too many loci,0.08
3,% of reads unmapped: too many mismatches,0.00
4,% of reads unmapped: too short,89.38
5,% of reads unmapped: other,0.12


## Generate plotly plot

### Subtask:
Create a bar chart or a pie chart using Plotly to visualize the different mapping categories (uniquely mapped, multi-mapped, unmapped).


**Reasoning**:
Generate a pie chart using Plotly to visualize the mapping statistics stored in the `mapping_df` DataFrame.



In [18]:
import plotly.express as px

fig = px.pie(mapping_df, values='Percentage', names='Category', title='STAR Alignment Mapping Statistics')
fig.show()

## Display plot

### Subtask:
Display the generated Plotly plot in the notebook.


**Reasoning**:
Display the generated Plotly plot in the notebook.



In [19]:
fig.show()

## Summary:

### Data Analysis Key Findings

*   The STAR alignment log file was successfully parsed to extract mapping statistics.
*   The extracted statistics were organized into a pandas DataFrame, including categories like "Uniquely mapped reads %", "% of reads mapped to multiple loci", and various unmapped categories.
*   A Plotly pie chart was generated to visualize the distribution of reads across these mapping categories.

### Insights or Next Steps

*   The pie chart provides a clear visual overview of the STAR alignment efficiency, highlighting the proportion of reads in different mapping categories.
*   Analyze the percentages of unmapped reads to identify potential issues with read quality, reference genome, or alignment parameters.


Here is a detailed interpretation of the key metrics from the `star_output/SRR11548957_Log.final.out` file:

*   **Started job on**: Indicates the date and time when the STAR alignment job began.
*   **Started mapping on**: Indicates the date and time when the read mapping process started.
*   **Finished on**: Indicates the date and time when the entire STAR alignment job finished.
*   **Mapping speed, Million of reads per hour**: This metric shows the processing speed of STAR, indicating how many million reads were aligned per hour. A higher number indicates faster processing.
*   **Number of input reads**: The total number of reads provided as input to STAR.
*   **Average input read length**: The average length of the input reads.
*   **UNIQUE READS:** Section detailing statistics for reads that mapped to only one location in the reference genome.
*   **Uniquely mapped reads number**: The total count of reads that mapped uniquely to the genome.
*   **Uniquely mapped reads %**: The percentage of total input reads that mapped uniquely. This is a crucial metric for assessing the quality of the alignment. A higher percentage is generally better.
*   **Average mapped length**: The average length of the uniquely mapped reads.
*   **Number of splices: Total**: The total number of splice junctions detected by STAR.
*   **Number of splices: Annotated (sjdb)**: The number of detected splice junctions that match known splice junctions provided in the GTF annotation file.
*   **Number of splices: GT/AG**: The number of splice junctions with the canonical GT/AG intron motif. This is the most common type of splice junction.
*   **Number of splices: GC/AG**: The number of splice junctions with the GC/AG intron motif, another common type.
*   **Number of splices: AT/AC**: The number of splice junctions with the less common AT/AC intron motif.
*   **Number of splices: Non-canonical**: The number of splice junctions that do not match any of the canonical or common non-canonical motifs. A high number here might indicate issues with the data or alignment.
*   **Mismatch rate per base, %**: The percentage of bases that are mismatched between the reads and the reference genome. A low mismatch rate is desirable.
*   **Deletion rate per base**: The rate of deletions per base in the aligned reads compared to the reference.
*   **Deletion average length**: The average length of deletions.
*   **Insertion rate per base**: The rate of insertions per base in the aligned reads compared to the reference.
*   **Insertion average length**: The average length of insertions.
*   **MULTI-MAPPING READS:** Section detailing statistics for reads that mapped to more than one location in the reference genome.
*   **Number of reads mapped to multiple loci**: The total count of reads that mapped to multiple locations.
*   **% of reads mapped to multiple loci**: The percentage of total input reads that mapped to multiple locations.
*   **Number of reads mapped to too many loci**: The count of reads that mapped to more locations than the maximum allowed by STAR's parameters.
*   **% of reads mapped to too many loci**: The percentage of total input reads that mapped to too many locations.
*   **UNMAPPED READS:** Section detailing statistics for reads that did not map to the reference genome.
*   **Number of reads unmapped: too many mismatches**: The count of reads that did not map due to exceeding the allowed number of mismatches.
*   **% of reads unmapped: too many mismatches**: The percentage of total input reads that did not map due to too many mismatches.
*   **Number of reads unmapped: too short**: The count of reads that did not map because they were too short after trimming or soft clipping.
*   **% of reads unmapped: too short**: The percentage of total input reads that did not map because they were too short.
*   **Number of reads unmapped: other**: The count of reads that did not map for reasons other than too many mismatches or being too short.
*   **% of reads unmapped: other**: The percentage of total input reads that did not map for other reasons.
*   **CHIMERIC READS:** Section detailing statistics for chimeric reads (reads that map to two different locations).
*   **Number of chimeric reads**: The total count of chimeric reads detected.
*   **% of chimeric reads**: The percentage of total input reads that were identified as chimeric.

Here is a summary of the key findings from the STAR alignment of SRR11548957:

*   **Mapping Speed:** The alignment processed reads at a rate of 4.74 million reads per hour.
*   **Input Reads:** A total of 1,792,464 reads were input for alignment, with an average length of 150 bases.
*   **Uniquely Mapped Reads:** Only 8.76% of the reads (156,947) uniquely mapped to the reference genome (chromosome 22). This is a relatively low unique mapping rate, suggesting either the reads are from a different genome/chromosome, the reference genome is incomplete, or there are issues with the read quality or adapter trimming.
*   **Multi-mapping Reads:** 1.66% of reads (29,782) mapped to multiple locations, and a very small percentage (0.08%) mapped to too many loci.
*   **Unmapped Reads:** A large majority of reads (89.38%) were unmapped because they were too short. This is a significant percentage and might indicate issues with the quality of the input reads or the chosen alignment parameters. A small percentage (0.12%) were unmapped for other reasons.
*   **Splice Junctions:** A total of 12,073 splice junctions were detected, with a good proportion (6,261) being annotated in the provided GTF file. The majority of detected junctions have canonical GT/AG (10,520) and GC/AG (209) motifs, with a smaller number of non-canonical junctions (1337).
*   **Mismatch, Deletion, and Insertion Rates:** The mismatch rate per base was 5.16%, which is quite high. The deletion and insertion rates were lower, at 0.11% and 0.17% respectively.

**Overall Assessment:** The alignment had a very low unique mapping rate and a high percentage of reads unmapped due to being too short, as well as a high mismatch rate. This suggests potential issues with the input data quality or that the chosen reference (chromosome 22) may not be the most appropriate for these reads. Further investigation into the read quality and the expected origin of the sequencing data would be recommended.

**Conclusion of the RNA-Seq Alignment Workflow:**

This notebook demonstrates a basic RNA-Seq alignment workflow using STAR, focusing on a subset of data (SRR11548957) and a specific reference genome (human chromosome 22). The steps performed were:

1.  **Environment Setup:** Installed necessary tools and libraries including `sra-toolkit` for data download and `STAR` for alignment.
2.  **Data Download:** Downloaded paired-end FASTQ files for the accession SRR11548957 from the SRA database.
3.  **Reference Preparation:** Downloaded and uncompressed the reference genome sequence (chromosome 22) and the corresponding GTF annotation file from Ensembl.
4.  **Genome Indexing:** Generated a STAR genome index using the downloaded reference genome and GTF file. This is a crucial step for efficient alignment.
5.  **Read Alignment:** Aligned the downloaded FASTQ reads to the generated STAR index, producing a sorted BAM file and gene counts.
6.  **Output Inspection and Download:** Inspected key output files (`Aligned.sortedByCoord.out.bam`, `ReadsPerGene.out.tab`, and `Log.final.out`) and provided code to download them.

**Summary of Findings:**

Based on the `Log.final.out` analysis, while the alignment successfully ran, the unique mapping rate was low (8.76%), and a high percentage of reads (89.38%) were unmapped due to being too short. The mismatch rate was also relatively high (5.16%). This suggests that the reads might not be primarily from chromosome 22 or that there are quality issues with the sequencing data.

**Next Steps:**

For a complete RNA-Seq analysis, the next steps would typically involve:

*   More in-depth quality control of the raw reads (e.g., using FastQC and MultiQC before alignment).
*   Considering aligning to the full human genome for a more comprehensive analysis.
*   Performing differential gene expression analysis using the gene counts obtained from STAR and appropriate statistical software (e.g., DESeq2, edgeR).
*   Further downstream analysis and visualization of the results.